In [ ]:
##Experimento 02
## En este experimento no se agregan los identificadores de arasaac al modelo

In [1]:
!python --version

import torch
print(torch.__version__)
print(torch.cuda.is_available())

!pip install transformers==4.50.0
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate==0.4.0
!pip install bleu
!pip install -U scikit-learn
!pip install nltk datasets
!pip install sacrebleu

Python 3.10.15
2.5.1+cu118
True
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import torch
import numpy as np
import nltk
import evaluate
import sacrebleu

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

os.environ["CUDA_VISIBLE_DEVICES"]="1,3"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

torch.cuda.empty_cache()


In [3]:
#import json
#lista_pictogramas_ids=[]
#with open("ids_arasaac.txt", 'r', encoding='utf-8') as archivo:
#  lineas = archivo.readlines()
#  for linea in lineas:
#    lista_pictogramas_ids.append(linea.strip())

#print(len(lista_pictogramas_ids))

In [4]:
dataset_train = load_dataset('json', data_files='train_data.json')['train']
dataset_test = load_dataset('json', data_files='test_data.json')['train']
dataset_valid = load_dataset('json', data_files='validation_data.json')['train']

# Mostrar estadísticas básicas
print(dataset_train)
print(dataset_test)
print(dataset_valid)

Dataset({
    features: ['id', 'oracion', 'traduccion'],
    num_rows: 15362
})
Dataset({
    features: ['id', 'oracion', 'traduccion'],
    num_rows: 423
})
Dataset({
    features: ['id', 'oracion', 'traduccion'],
    num_rows: 422
})


In [5]:
model_checkpoint = "flax-community/spanish-t5-small" #"vgaraujov/t5-base-spanish"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")

total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

# Agregar nuevos tokens al tokenizer
#existing_tokens = set(tokenizer.get_vocab().keys())
#new_tokens = [token for token in lista_pictogramas_ids if token not in existing_tokens]
#tokenizer.add_tokens(new_tokens)

#model.resize_token_embeddings(len(tokenizer))

# Validar parámetros actualizados
#print("After adding new tokens:")
#total_params = sum(p.numel() for p in model.parameters())
#print(f"{total_params:,} total parameters.")
#total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#print(f"{total_trainable_params:,} training parameters.")

#new_token_ids = tokenizer.convert_tokens_to_ids(new_tokens)
#print(f"Sample IDs of new tokens: {new_token_ids[:10]}")

#new_embeddings = model.get_input_embeddings().weight.data[new_token_ids]
#print("Sample embeddings for new tokens:", new_embeddings[:5])

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Convertir los nuevos tokens a sus IDs
#new_token_ids = tokenizer.convert_tokens_to_ids(new_tokens)

# Mostrar ejemplos de tokens agregados y sus IDs
#for token, token_id in zip(new_tokens[:10], new_token_ids[:10]):
#    print(f"Token: {token}, ID: {token_id}")

60,493,824 total parameters.
60,493,824 training parameters.


In [6]:
model.to(torch.device('cuda'))
model.device

device(type='cuda', index=0)

In [7]:
max_input_length = 40
max_target_length = 40

batch_size=16
metric ="bleu"
model_name = "t5-xgen-finetuned"
evaluation_strategy = "epoch"
save_strategy="epoch"
overwrite_output_dir=True
learning_rate=10e-5
gradient_accumulation_steps=1
weight_decay=0.01
do_train=True
do_eval=True
save_total_limit=20
num_train_epochs=20
seed=42
predict_with_generate=True
fp16=True
metric_for_best_model="bleu"
load_best_model_at_end=True
generation_max_length = max_target_length
logging_strategy="epoch"
eval_accumulation_steps=2

In [8]:
def preprocess_function(examples):
    inputs = ["translate: " + oracion for oracion in examples['oracion']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")    
    labels = tokenizer(text_target=examples["traduccion"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
encoded_train = dataset_train.map(preprocess_function, batched=True)
encoded_test = dataset_test.map(preprocess_function, batched=True)
encoded_validation = dataset_valid.map(preprocess_function, batched=True)

In [10]:
nltk.download("punkt", quiet=True)

bleu_metric = evaluate.load("bleu")
chrf_metric = evaluate.load('chrf')

def compute_metrics_v2(eval_preds):
    print(f"metrics v2")
    predictions, labels = eval_preds

    # Reemplazar -100 con pad_token_id tanto en predicciones como en etiquetas
    pad_token_id = tokenizer.pad_token_id
    predictions = np.where(predictions != -100, predictions, pad_token_id)
    labels = np.where(labels != -100, labels, pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
     
    # Asegurarse de que BLEU reciba cadenas completas, no listas de palabras
    preds = [" ".join(pred.split()) for pred in decoded_preds]
    refs = [[" ".join(label.split())] for label in decoded_labels]  # Lista de listas para referencias
    
    print(f"bleu_preds {preds}")
    print(f"bleu_refs {refs}")
    
    # Calcular BLEU
    try:
        bleu_result = bleu_metric.compute(predictions=preds, references=refs)
        bleu_score = {"bleu": bleu_result["bleu"], "precisions": bleu_result["precisions"]}
    except Exception as e:
        print(f"Error al calcular BLEU: {e}")
        bleu_score = {"bleu": 0.0}

    # Calcular CHRF++
    try:
        chrf_result = chrf_metric.compute(predictions=preds, references=refs)
        print(chrf_result)
        chrf_score = {"chrf": chrf_result["score"]}
    except Exception as e:
        print(f"Error al calcular CHRF++: {e}")
        chrf_score = {"chrf": 0.0}
        
    # Combinar resultados
    combined_results = {**bleu_score, **chrf_score}
    return combined_results

In [11]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy = evaluation_strategy,
    save_strategy = save_strategy,
    overwrite_output_dir = overwrite_output_dir,
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size= batch_size,
    gradient_accumulation_steps= gradient_accumulation_steps,
    weight_decay= weight_decay,
    do_train= do_train,
    do_eval= do_eval,
    save_total_limit= save_total_limit,
    num_train_epochs= num_train_epochs,
    seed= seed,
    predict_with_generate= predict_with_generate,
    fp16= fp16,
    generation_max_length=generation_max_length,
    logging_strategy=logging_strategy,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_v2
)



C:\ProgramData\anaconda3\envs\thesis\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\apare\AppData\Local\Temp\ipykernel_7524\3537574832.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [12]:
import numpy as np

import nltk
nltk.download('punkt')

trainer.train()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\apare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Precisions,Chrf
1,0.764500,1.230117,0.381607,"[0.720937218383899, 0.4879559532002753, 0.284967845659164, 0.23288331726133077]",52.622111
2,0.362200,1.275945,0.418487,"[0.736257134274557, 0.5165175498967653, 0.3216867469879518, 0.27601156069364163]",55.227876
3,0.251100,1.300148,0.442614,"[0.7500752785305631, 0.5403726708074534, 0.3527990334273057, 0.29840656687590533]",57.734154
4,0.189700,1.371757,0.464493,"[0.7606015037593985, 0.5582356995175741, 0.37771520514883344, 0.32111861137897785]",59.628425
5,0.147600,1.359697,0.464586,"[0.7588502269288956, 0.5593337959750173, 0.3808174828004856, 0.3268765133171913]",59.432695
6,0.115900,1.496038,0.472672,"[0.763614237906906, 0.5663407821229051, 0.39135752140236446, 0.3421309872922776]",59.988592
7,0.093600,1.533871,0.482796,"[0.7639472105578884, 0.5695637238062522, 0.3941459502806736, 0.3466346153846154]",60.959254
8,0.075100,1.498066,0.484770,"[0.7645470905818836, 0.5705942974922708, 0.3957497995188452, 0.35]",60.806327
9,0.061000,1.586655,0.488961,"[0.7726035681886907, 0.5794036061026352, 0.40728744939271255, 0.3546821931101407]",61.489363
10,0.049700,1.611909,0.491924,"[0.7748264412918805, 0.5823529411764706, 0.41057731126362534, 0.35493230174081236]",61.815138


Trainer is attempting to log a value of "[0.720937218383899, 0.4879559532002753, 0.284967845659164, 0.23288331726133077]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


metrics v2
bleu_preds ['pict_2472_calima pict_2', 'pict_2474_yo borro pict_7', 'pict_3414_¡ pict_22624', 'pict_3415_¿ pict_7764_', 'pict_11749_hacer pict_8474', 'pict_11702_relacionar pict_', 'pict_12268_tu pict_2262', 'pict_6903_animal pict_70', 'pict_2474_observar pict_7', 'pict_3415_¿ pict_7764_', 'pict_2476_alimentar pict_30', 'pict_2474_observar pict_7', 'pict_2381_escuchar pict_7029', 'pict_8474_explicar pict_847', 'pict_3415_¿ pict_7212_', 'pict_2474_observar pict_8', 'pict_11308_mostrar pict_84', 'pict_2474_yo pict_2476', 'pict_2474_yo pict_8476', 'pict_3415_¿ pict_8474_', 'pict_2728_dos geométricos', 'pict_2619_me pict_2619', 'pict_2629_población pict_7034', 'pict_2474_observar pict_8', 'pict_2692_animal pict_2', 'pict_13028_antes pict_7074', 'pict_2772_cáscara pict_7074', 'pict_6481_lo pict_1174', 'pict_6483_elegir pict_247', 'responder pict_7029_la pict_702', 'evita distraerte', 'pict_2474_mirar pict_1226', 'pict_2381_yo pict_2476', 'pict_3345_hablar pict_7074', 'pict_2381_y

Trainer is attempting to log a value of "[0.736257134274557, 0.5165175498967653, 0.3216867469879518, 0.27601156069364163]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


...

metrics v2
bleu_preds ['pict_2709_canción pict_46', 'pict_2617_yo borro pict_7', 'pict_3414_¡ pict_12268', 'pict_3415_¿ pict_7764_', 'pict_11749_hacer pict_8474', 'pict_6192_proteger pict_847', 'pict_12281_tu pict_7151', 'pict_7775_trabajadores pict_7034', 'pict_2474_observar pict_7', 'pict_3415_¿ pict_7764_', 'pict_7245_siembra pict_3', 'pict_2474_observar pict_5', 'pict_2381_escuchar pict_7029', 'pict_8579_explicar pict_847', 'pict_3415_¿ pict_7212_', 'pict_2474_observar pict_8', 'pict_6998_revisar pict_8', 'pict_2617_yo pict_5467', 'pict_2617_yo pict_7141', 'pict_3415_¿ pict_15485', 'pict_2728_cuerpo geométricos', 'pict_2617_me pict_2418', 'pict_14559_ pobreza pict_7034', 'pict_2474_observar pict_8', 'pict_9002_turismo pict_8239', 'pict_13028_antes pict_7074', 'pict_24863_ceja pict_7', 'pict_6481_lo pict_1174', 'pict_6483_elegir pict_262', 'pict_9030_responder pict_70', 'pict_24031_evitar', 'pict_2474_mirar pict_1227', 'pict_2617_yo pict_5441', 'pict_3345_hablar pict_7074', 'pict_26

Trainer is attempting to log a value of "[0.7748045700541191, 0.5852566310713055, 0.4153598713309208, 0.3642064640617463]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


metrics v2
bleu_preds ['pict_2712_ luna pict_4686_', 'pict_2617_yo borro pict_7', 'pict_3414_¡ pict_12268', 'pict_3415_¿ pict_7764_', 'pict_11749_hacer pict_8474', 'pict_6192_proteger pict_847', 'pict_12281_tu pict_7151', 'pict_7795_trabajadores pict_7034', 'pict_2474_observar pict_7', 'pict_3415_¿ pict_7764_', 'pict_7245_siembra pict_3', 'pict_2474_observar pict_5', 'pict_2381_escuchar pict_7029', 'pict_8579_explicar pict_847', 'pict_3415_¿ pict_7212_', 'pict_2474_observar pict_8', 'pict_6998_revisar pict_8', 'pict_2617_yo pict_5467', 'pict_2617_yo pict_7141', 'pict_3415_¿ pict_15485', 'pict_2728_cuerpo geométricos', 'pict_2617_me pict_2418', 'pict_14559_ pobreza pict_7034', 'pict_2474_observar pict_8', 'pict_9002_turismo pict_8239', 'pict_13028_antes pict_7074', 'pict_24863_ceja pict_7', 'pict_6481_lo pict_1174', 'pict_6483_elegir pict_262', 'pict_9030_responder pict_70', 'pict_24031_evitar', 'pict_2474_mirar pict_1227', 'pict_2617_yo pict_5441', 'pict_3345_hablar pict_7074', 'pict_2

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=19220, training_loss=0.11546654882341717, metrics={'train_runtime': 23801.349, 'train_samples_per_second': 12.909, 'train_steps_per_second': 0.808, 'total_flos': 1624313089228800.0, 'train_loss': 0.11546654882341717, 'epoch': 20.0})

In [ ]:
## Se necesita saber con exactitud como se llama el folder para cargar el mejor modelo

tokenizerv1 = AutoTokenizer.from_pretrained('./results/checkpoint-2196')
modelv1 = AutoModelForSeq2SeqLM.from_pretrained('./results/checkpoint-2196')

device = torch.device("cuda")
modelv1.to(device)
modelv1.eval()

In [ ]:
!pip install matplotlib


# Función para preparar los datos para la predicción
def prepare_data_for_prediction(dataset, tokenizer):
    # Transformar los datos para que sean aptos para el modelo
    prepared_data = []
    for example in dataset:
        inputs = tokenizerv1("translate: " + example['oracion'], return_tensors="pt", padding="max_length", truncation=True, max_length=max_input_length)
        inputs = {k: v.to(device1) for k, v in inputs.items()}
        prepared_data.append(inputs)
    return prepared_data

# Preparar los datos de validación para la predicción
prepared_validation_data = prepare_data_for_prediction(dataset_valid, tokenizerv1)

# Lista para guardar las predicciones
predictions = []

# Generar predicciones para cada entrada en los datos de validación
for inputs in prepared_validation_data:
    with torch.no_grad():
        outputs = modelv1.generate(**inputs, max_length=max_target_length)
    decoded_output = tokenizerv1.decode(outputs[0], skip_special_tokens=True)
    predictions.append(decoded_output)

# Imprimir las predicciones
for i, prediction in enumerate(predictions):
    print(f"Oracion: {dataset_valid[i]}, Prediction {i+1}: {prediction}")